# How to transform shapes in 2D...
## ... with the little help of linear algebra!

In [ ]:
from numpy import array, identity as I
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from math import pi as π, cos, sin
from random import random

# A bit of randomness...
σ = lambda x = .5: x*(random() - 0.5)

# Transformation of the polygon's vertices
transform = lambda P, points: array([P @ p for p in points])
# Homogeneous to Euclidean coordinates conversion
h2e = lambda X: array([(x/x[-1])[:-1] for x in X])

### A square (a kinky one...) 

Various $3\times 3$ matrices
$$
\mathbf{I}=
\begin{bmatrix}
1 & 0 & 0\\
0 & 1 & 0\\
0 & 0 & 1
\end{bmatrix}
$$
transform vectors $\left[x, y, 1\right]^T$ of 2D point in homogenous coordinates (this matrix does nothing).

In [ ]:
# The original shape at the origin
sqwk = array([[0, 0, 1], [1, 0, 1], [1, 1, 1], [.25, .25, 1], [0, 1, 1]])

# Identity transformation (a.k.a. "do nothing but with great style!")
sqwk = transform(I(3), sqwk)
verticesI = h2e(sqwk)

## Translation/shift

Thanks to the homogeneous coordinates, translation by $\left({\color{red}{K}}, {\color{blue}{L}}\right)$ is a linear operation!
$$
\mathbf{T}_{{\color{red}{K}}, {\color{blue}{L}}}=
\begin{bmatrix}
1 & 0 & {\color{red}{K}}\\
0 & 1 & {\color{blue}{L}}\\
0 & 0 & 1
\end{bmatrix}
$$

In [ ]:
# Our square after translation by K and L...
K, L = 1.5 + σ(), 1 + σ(2)
T = I(3); _T = array([K, L])
T[:2, 2] = _T.T # Transposition seems superfluous here, but it's a good habit

## Rotation

Here is a formula of a rotation matrix (by an angle ${\color{red}{\alpha}}$)
$$
\mathbf{R}_{\color{red}{\alpha}}=
\begin{bmatrix}
\cos{\color{red}{\alpha}} & -\sin{\color{red}{\alpha}} & 0\\
\sin{\color{red}{\alpha}} & \cos{\color{red}{\alpha}} & 0\\
0 & 0 & 1
\end{bmatrix}
$$
Observe that $\det(R) = \cos^2{\color{red}{\alpha}} + \sin^2{\color{red}{\alpha}} = 1$:

In [ ]:
# ... and after rotation by α degrees...
α = σ(30)*π/180
R = I(3); _R = array([[cos(α), -sin(α)],
                      [sin(α),  cos(α)]])
R[:2, :2] = _R

## Squeezing, stretching, shearing & scaling section – [Procrustes](https://en.wikipedia.org/wiki/Procrustes) would've loved it!

These matrices look s's''s'''straighforward:
$$
\mathbf{S}'_{{{\color{red}{k}},{\color{blue}{\kappa}}}}=
\begin{bmatrix}
{\color{red}{k}} & 0 & 0\\
0 & {\color{blue}{\kappa}} & 0\\
0 & 0 & 1
\end{bmatrix},\quad
\mathbf{S''_{\color{blue}{\kappa}}}=
\begin{bmatrix}
{\color{blue}{\kappa}} & 0           & 0\\
0      & {\color{blue}{\kappa}}^{-1} & 0\\
0 & 0 & 1
\end{bmatrix},\quad
\mathbf{S}'''_{{{\color{red}{k}},{\color{blue}{\kappa}}}}=
\begin{bmatrix}
1      & {\color{red}{k}} & 0\\
{\color{blue}{\kappa}} & 1      & 0\\
0 & 0 & 1
\end{bmatrix},\quad
\mathbf{S}_{{\color{red}s}}=
\begin{bmatrix}
{\color{red}{s}} & 0 & 0\\
0 & {\color{red}{s}} & 0\\
0 & 0 & 1
\end{bmatrix}
$$

In [ ]:
#  See https://en.wikipedia.org/wiki/Transformation_matrix#/media/File:2D_affine_transformation_matrix.svg
#  and https://en.wikipedia.org/wiki/Transformation_matrix#/media/File:Perspective_transformation_matrix_2D.svg

# stretching
k, κ = 2 + σ(), 2 + σ()
S1 = array([[k, 0, 0],
            [0, κ, 0],
            [0, 0, 1]])
# ... squeezing
S2 = array([[1/κ, 0, 0],
            [0,   κ, 0],
            [0,   0, 1]])
# and shearing
S3 = array([[1, k, 0],
            [κ, 1, 0],
            [0, 0, 1]])
# ... and just scaling, too
s = σ(2)
S = I(3); _S = array([[s, 0],
                      [0, s]])
S[:2, :2] = _S

## Reflections (ThnX, [Hauseholder](https://en.wikipedia.org/wiki/Alston_Scott_Householder)!)

It is not exactly obvious that this matrix implements reflection about the line from origin $(0, 0)$ through $({\color{red}{l_x}}, {\color{blue}{l_y}})$. 
It will be clear, however, once we derive a projection formula
$$
\mathbf{M}_{{\color{red}{l_x}},{\color{blue}{l_y}}}=\frac{1}{{\color{red}{l_x}}^2 + {\color{blue}{l_y}}^2}
\begin{bmatrix}
{\color{red}{l_x}}^2 - {\color{blue}{{l_y}}}^2 & 2{\color{red}{l_x}}\color{blue}{{l_y}} & 0\\
{2\color{red}{l_x}}\color{blue}{{l_y}} & {\color{red}{l_x}}^2 - {\color{blue}{{l_y}}}^2 & 0\\
0 & 0 & 1
\end{bmatrix}
$$

In [ ]:

# Reflecting:
lx, ly = .5 + σ(), .5 + σ()
M = I(3)
_M = array([[lx**2 - ly**2,     2*lx*ly  ],
            [    2*lx*ly  , ly**2 - lx**2]])
_M = _M/(lx**2 + ly**2)
M[:2, :2] = _M

## All in a single step!

Regardless of how many transformation we need to apply, we can always multiply them first to get a single transformation matrix
$$
X' = {\color{red}{\Theta}} X,\quad {\color{red}{\Theta}} = \mathbf{M}_{{\color{red}{l_x}},{\color{blue}{l_y}}}\cdot\mathbf{T}_{{\color{red}{K}},{\color{blue}{L}}} \cdot\mathbf{S}_{{\color{red}s}}\cdot\mathbf{R}_{\color{red}{\alpha}}\cdot\mathbf{S}'''_{{{\color{red}{k}},{\color{blue}{\kappa}}}}
$$
Remember, however, about the order: matrix multiplication is not commutative (in general).

In [ ]:
# Feel free to further squeeze, stretch, shear the shape (and mix the order too)...
Θ = M@T@S@R@S3  # M@T@S@R@S3 != S3@R@S@T@M, for instance!
sqwk = transform(Θ, sqwk)
verticesII = h2e(sqwk)


# Projections
Perhaps surprisingly, given a form of the matrix of Hausholder, that perspective projection matrix is fairly simple. 
$$
\mathbf{P}_{{\color{red}{A}}, {\color{blue}{B}}}=
\begin{bmatrix}
1 & 0 & 0\\
0 & 1 & 0\\
{\color{red}{A}} & {\color{blue}{B}} & 0
\end{bmatrix}
$$
and turns a homogeneous point $X'$ into its projection $X''$ 
$$X'' = \mathbf{P}_{{\color{red}{A}}, {\color{blue}{B}}}X'$$
onto a line ${\color{red}{A}}x + {\color{blue}{B}}y = 1$ (with a center of the projection located at the origin $(0, 0)$).

In [ ]:
# Set the stage!
# https://www.geeksforgeeks.org/how-to-draw-shapes-in-matplotlib-with-python/
fig, ax = plt.subplots()
ax.set_xlim(-1, 4); ax.set_ylim(-1, 4); ax.set_aspect('equal'); plt.grid(True)

# Eventually, the final projection (a.k.a. casting a shadow)...
# https://youtu.be/27vT-NWuw0M, https://youtu.be/JK-8XNIoAkI and https://youtu.be/cTyNpXB92bQ
# which is a linear operation too and can be represented by a matrix as well:
# https://wrfranklin.org/pmwiki/Main/HomogeneousCoords
# The shadow of the final shape on the line Ax + By = 1
A, B = 1 + σ(), 1 + σ()
P = array([[1, 0, 0],
           [0, 1, 0],
           [A, B, 0]])
shdw = transform(P, sqwk); verticesIII = h2e(shdw)

# https://stackoverflow.com/questions/44526364/fill-matplotlib-polygon-with-a-gradient-between-vertices
for (vertices, edges, fills) in [(verticesI, 'crimson', 'orange'),
                                 (verticesII, 'maroon', 'orangered'),
                                 (verticesIII, 'brown', 'red')]:
    polygon = Polygon(vertices, ec = edges, fc = fills, alpha = 0.5)
    ax.scatter(vertices[:, 0], vertices[:, 1], color = edges, s = 25)
    ax.add_patch(polygon)


# And the ♪♫Moonlight shadows♫♪... https://youtu.be/ixExC-Zgyzc and https://youtu.be/e80qhyovOnA of the vertices.
# Is that song about Évariste Galois https://en.wikipedia.org/wiki/%C3%89variste_Galois#Final_days?
for v, w in zip(sqwk, shdw):
    # Homogeneous to Euclidean coordinates
    (x, y), (u, r) = v[:-1]/v[-1], w[:-1]/w[-1]
    ax.plot((0, x), (0, y), c = 'tan', lw = .5, ls = 'dashed')
    ax.plot((0, u), (0, r), c = 'goldenrod', lw = .5, ls = 'dotted')

plt.title('3M@2D\n(marvelous matrix multiplications in two dimensions)\nrigid, linear, affine & projective transformations'); plt.show()